<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_02_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

In [2]:
assert ('google.colab' in sys.modules), "ERROR - the script expects to be run in Colab"

In [3]:
# we persist code, dataset and runs on google drive...
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
work_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/'
os.chdir(work_folder)
print("checking git status (git fetch --dry-run):")
!pwd
!git fetch --dry-run

checking git status (git fetch --dry-run):
/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), 262.44 KiB | 367.00 KiB/s, done.
From https://github.com/cwinsor/medical_image_uw_madison
   18a51f5..4418c42  main       -> origin/main


In [ ]:
try:
    sys.path.index('/dir/path') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('/dir/path') # Or os.getcwd() for this directory

!echo #PATH

In [16]:
paths = os.getenv('PATH')
print(type(paths))
for p in paths.split(":"):
    print(p)

<class 'str'>
/opt/bin
/usr/local/nvidia/bin
/usr/local/cuda/bin
/usr/local/sbin
/usr/local/bin
/usr/sbin
/usr/bin
/sbin
/bin
/tools/node/bin
/tools/google-cloud-sdk/bin


In [15]:
THE_FOLDER = "foo"
# THE_FOLDER = os.getcwd()

try:
    sys.path.index(THE_FOLDER)
    print("already exists")
except ValueError:
    sys.path.append(THE_FOLDER)
    print("added")

already exists


In [17]:
paths = os.getenv('PATH')
print(type(paths))
for p in paths.split(":"):
    print(p)

<class 'str'>
/opt/bin
/usr/local/nvidia/bin
/usr/local/cuda/bin
/usr/local/sbin
/usr/local/bin
/usr/sbin
/usr/bin
/sbin
/bin
/tools/node/bin
/tools/google-cloud-sdk/bin


In [ ]:
# train classification
CONFIG = 'work_configs/tract/final_solution/classification_configs/cls_1.py'
GPUS = 2
PORT =

PYTHONPATH="$(dirname $0)/..":$PYTHONPATH \

python -m torch.distributed.launch --nproc_per_node=$GPUS --master_port=$PORT \
    $(dirname "$0")/train.py $CONFIG --launcher pytorch ${@:3}

!

In [5]:
# now... from bash shell (bottom left in colab notebook)
# cd /content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project
# ./go_02_train.sh